# Analyse Avancée des Sentiments avec des Modèles LSTM

Ce notebook a pour objectif de développer et comparer des modèles avancés pour l'analyse des sentiments appliquée à des tweets.
Il constitue une avancée par rapport à l'approche de base utilisant une régression logistique, en explorant des architectures de deep learning plus complexes.

## Objectifs :
1. Mettre en œuvre et entraîner des modèles **Long Short-Term Memory (LSTM)** pour la classification des sentiments.
2. Évaluer les performances de deux méthodes d’embedding :
   - **Word2Vec** : Embeddings pré-entraînés capturant le contexte et la signification des mots.
   - **FastText** : Embeddings pré-entraînés intégrant des informations sur les sous-mots, utiles pour les mots rares ou mal orthographiés.
3. Suivre et analyser les performances des modèles grâce à **MLFlow**, en se concentrant sur des métriques telles que :
   - Précision (Validation Accuracy)
   - Perte (Validation Loss)
   - ROC-AUC

## Jeu de données et déroulement :
- **Jeu de données** : Dataset de tweets prétraités contenant des labels binaires (positif ou négatif).
- **Étapes** :
  1. Tokenisation et padding des tweets.
  2. Intégration des embeddings Word2Vec et FastText pré-entraînés.
  3. Construction et entraînement des modèles LSTM.
  4. Évaluation et comparaison des modèles à l’aide des données de validation et de test.

Ce notebook reflète une approche professionnelle et avancée pour les tâches de classification de texte en exploitant des techniques modernes de NLP.

In [2]:
# Manipulation de données et calculs
import pandas as pd
import numpy as np
import time
import os

# Outils pour la gestion des ensembles de données et l'évaluation des modèles
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Bibliothèques pour la construction et l'entraînement des modèles
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras_tuner as kt

# Chargement des embeddings pré-entraînés (Word2Vec, FastText)
from gensim.models import KeyedVectors

# Suivi et enregistrement des expérimentations avec MLFlow
import mlflow
import mlflow.keras
from mlflow.models.signature import infer_signature

In [4]:
# Charger le dataset prétraité
df = pd.read_csv('../data/processed_tweets.csv')

In [5]:
# Diviser les données
X = df['text']
y = df['sentiment']

# Tokenizer les tweets
tokenizer = Tokenizer(num_words=20000)  # Limiter à 20 000 mots
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=100)  # Fixer une longueur maximale

## Division des données en ensembles d'entraînement, validation et test

Afin de garantir une évaluation rigoureuse et éviter tout biais, les données sont divisées en trois ensembles distincts :

1. **Jeu de Test** :
   - 20% des données sont réservées pour le test final du modèle.
   - Ce jeu de données ne sera utilisé qu’à la fin pour évaluer objectivement les performances.

2. **Jeu de Validation** :
   - Sur les 80% restants (le jeu d’entraînement complet), 20% sont extraits pour constituer le jeu de validation.
   - Le jeu de validation est utilisé pendant l’entraînement pour ajuster les hyperparamètres et surveiller les performances sans sur-apprendre sur les données d’entraînement.

3. **Jeu d’Entraînement Final** :
   - Les 64% restants des données initiales sont utilisés pour entraîner le modèle.

Cette double division permet de maintenir une distinction claire entre les données utilisées pour l’entraînement, la validation et le test, en respectant les bonnes pratiques pour une évaluation fiable.

In [6]:
# Division en jeu d'entraînement, validation et test
X_train_full, X_test, y_train_full, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

In [8]:
# Charger Word2Vec pré-entraîné
word2vec = KeyedVectors.load_word2vec_format('../models/GoogleNews-vectors-negative300.bin', binary=True)

# Créer une matrice d'embedding pour Word2Vec
embedding_matrix_w2v = np.zeros((20000, word2vec.vector_size))
for word, i in tokenizer.word_index.items():
    if i < 20000 and word in word2vec:
        embedding_matrix_w2v[i] = word2vec[word]

In [9]:
# Charger FastText pré-entraîné
fasttext = KeyedVectors.load_word2vec_format('../models/crawl-300d-2M-subword.vec')

# Créer une matrice d'embedding pour FastText
embedding_matrix_ft = np.zeros((20000, fasttext.vector_size))
for word, i in tokenizer.word_index.items():
    if i < 20000 and word in fasttext:
        embedding_matrix_ft[i] = fasttext[word]

## Construction du modèle LSTM

La fonction `create_lstm_model` construit un modèle basé sur une architecture **LSTM** pour l'analyse des sentiments. 

- **Embedding** : 
  - Intègre une matrice d'embedding pré-entraînée (Word2Vec ou FastText) pour représenter les mots dans un espace vectoriel.
  - Les poids de la matrice sont fixés (`trainable=False`) pour conserver les embeddings d'origine.

- **LSTM Layer** :
  - Une couche LSTM (Long Short-Term Memory) est utilisée pour capturer les dépendances contextuelles dans les tweets.

- **Dense Layer** :
  - Une couche de sortie entièrement connectée avec une activation `sigmoid` pour produire une probabilité binaire (positif ou négatif).

- **Compilation** :
  - Le modèle est compilé avec l'optimiseur `adam` et une fonction de perte adaptée à la classification binaire (`binary_crossentropy`).

Cette fonction permet de construire dynamiquement des modèles adaptés à différentes matrices d’embedding.

In [10]:
def create_lstm_model(embedding_matrix):
    model = Sequential([
        Embedding(input_dim=20000, output_dim=embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False),
        LSTM(128, dropout=0.2, recurrent_dropout=0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
mlflow.set_experiment("Sentiment_Analysis_Advanced_Model")

for embedding_name, embedding_matrix in [("Word2Vec", embedding_matrix_w2v), ("FastText", embedding_matrix_ft)]:
    with mlflow.start_run():
        # Loguer les hyperparamètres communs
        mlflow.log_param("embedding", embedding_name)
        mlflow.log_param("batch_size", 32)
        mlflow.log_param("epochs", 5)
        mlflow.log_param("max_sequence_length", 100)

        # Créer le modèle
        model = Sequential([
            Embedding(input_dim=20000, output_dim=embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False),
            LSTM(128, dropout=0.2, recurrent_dropout=0.2),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Chronométrer l'entraînement
        start_time = time.time()

        # Entraîner le modèle
        history = model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val))

        elapsed_time = time.time() - start_time

        # Calculer les métriques finales
        val_accuracy = history.history['val_accuracy'][-1]
        val_loss = history.history['val_loss'][-1]
        roc_auc = roc_auc_score(y_val, model.predict(X_val))

        # Loguer les métriques
        mlflow.log_metric("val_accuracy", val_accuracy)
        mlflow.log_metric("val_loss", val_loss)
        mlflow.log_metric("roc_auc", roc_auc)
        mlflow.log_metric("training_time_seconds", elapsed_time)

        # Inférer la signature
        signature = infer_signature(X_val, model.predict(X_val))

        try:
            # Enregistrer le modèle directement dans MLFlow
            mlflow.keras.log_model(
                model=model,
                artifact_path="model",
                signature=signature
            )
            print(f"Modèle {embedding_name} enregistré avec succès dans MLFlow.")
        except Exception as e:
            print(f"Erreur lors de l'enregistrement du modèle {embedding_name} : {e}")

2024/11/19 15:34:09 INFO mlflow.tracking.fluent: Experiment with name 'Sentiment_Analysis_Advanced_Model' does not exist. Creating a new experiment.


Epoch 1/5
31867/31867 ━━━━━━━━━━━━━━━━━━━━ 1018s 32ms/step - accuracy: 0.7480 - loss: 0.5072 - val_accuracy: 0.7852 - val_loss: 0.4528
Epoch 2/5
31867/31867 ━━━━━━━━━━━━━━━━━━━━ 994s 31ms/step - accuracy: 0.7850 - loss: 0.4536 - val_accuracy: 0.7917 - val_loss: 0.4437
Epoch 3/5
31867/31867 ━━━━━━━━━━━━━━━━━━━━ 1006s 32ms/step - accuracy: 0.7927 - loss: 0.4413 - val_accuracy: 0.7937 - val_loss: 0.4396
Epoch 4/5
31867/31867 ━━━━━━━━━━━━━━━━━━━━ 998s 31ms/step - accuracy: 0.7956 - loss: 0.4361 - val_accuracy: 0.7948 - val_loss: 0.4385
Epoch 5/5
31867/31867 ━━━━━━━━━━━━━━━━━━━━ 1010s 32ms/step - accuracy: 0.7994 - loss: 0.4297 - val_accuracy: 0.7958 - val_loss: 0.4365
7967/7967 ━━━━━━━━━━━━━━━━━━━━ 86s 11ms/step
7967/7967 ━━━━━━━━━━━━━━━━━━━━ 83s 10ms/step
Modèle Word2Vec enregistré avec succès dans MLFlow.
Epoch 1/5
31867/31867 ━━━━━━━━━━━━━━━━━━━━ 892s 28ms/step - accuracy: 0.7518 - loss: 0.5053 - val_accuracy: 0.7868 - val_loss: 0.4507
Epoch 2/5
31867/31867 ━━━━━━━━━━━━━━━━━━━━ 870s 27m